In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from src.utils import *
from src.calcEMA import *
from src.robo import *
from src.myenv import *
import plotly.express as px

In [2]:
# Variables
# ETCUSDT BTCUSDT
# symbol = 'ETHUSDT'
symbol = 'BTCUSDT'
# lightgbm  xgboost
estimator = 'xgboost'
_compare_models = False

start_train_date = '2010-01-01'  # train < and test >=
start_test_date = '2023-01-01'  # train < and test >=

stop_loss = 2.0
label = 'status'
regression_times = 24 * 30 * 2  # horas
regression_profit_and_loss = 24
# numeric_features=['open', 'high', 'low', 'volume', 'close', 'rsi']
numeric_features = ['open', 'high', 'low', 'volume', 'close' ]
_calc_rsi = True


### Metadata

<code>
Field Name - Description</br>
open_time - Kline Open time in unix time format</br>
open - Open Price</br>
high - High Price</br>
low	- Low Price</br>
close	- Close Price</br>
volume - Volume</br>
close_time - Kline Close time in unix time format</br>
quote_volume - Quote Asset Volume</br>
count	- Number of Trades</br>
taker_buy_volume - Taker buy base asset volume during this period</br>
taker_buy_quote_volume - Taker buy quote asset volume during this period</br>
ignore - Ignore</br>
</code>

In [3]:
use_cols = date_features + numeric_features
print(use_cols)
all_data = read_data(f'{datadir}/{symbol}', all_cols=None, use_cols=use_cols)

# Filter all data
all_data = all_data[(all_data['open_time'] >= start_train_date)].copy()

print(all_data.info())

['open_time', 'open', 'high', 'low', 'volume', 'close']
read_data: Start reading file:  /home/marcelo/des/mg_crypto_trader/src/data/BTCUSDT/BTCUSDT.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 52920 entries, 0 to 52919
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   open_time  52920 non-null  datetime64[ns]
 1   open       52920 non-null  float64       
 2   high       52920 non-null  float64       
 3   low        52920 non-null  float64       
 4   close      52920 non-null  float64       
 5   volume     52920 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 2.8 MB
None


In [4]:
test_data = all_data[all_data['open_time'] >= start_test_date]
test_data

,open_time,open,high,low,close,volume
46981,2023-01-01 00:00:00,16541.77,16545.70,16508.39,16529.67,4364.83570
46982,2023-01-01 01:00:00,16529.59,16556.80,16525.78,16551.47,3590.06669
46983,2023-01-01 02:00:00,16551.47,16559.77,16538.14,16548.19,3318.84038
46984,2023-01-01 03:00:00,16548.19,16548.19,16518.21,16533.04,4242.08050
46985,2023-01-01 04:00:00,16533.04,16535.97,16511.92,16521.85,4285.00909
...,...,...,...,...,...,...
52915,2023-09-04 21:00:00,25836.68,25842.23,25631.21,25677.71,1856.10935
52916,2023-09-04 22:00:00,25677.72,25798.54,25632.61,25775.01,839.81115
52917,2023-09-04 23:00:00,25775.02,25874.97,25764.02,25826.02,1283.81847
52918,2023-09-05 00:00:00,25826.03,25844.48,25737.69,25768.49,860.37518


In [5]:
if _calc_rsi:
  test_data = calc_RSI(test_data)
  numeric_features.append('rsi')
  test_data.dropna(inplace=True)
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5925 entries, 46995 to 52919
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   open_time  5925 non-null   datetime64[ns]
 1   open       5925 non-null   float64       
 2   high       5925 non-null   float64       
 3   low        5925 non-null   float64       
 4   close      5925 non-null   float64       
 5   volume     5925 non-null   float64       
 6   rsi        5925 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 370.3 KB
None


In [6]:
experiment, model = load_model(symbol, estimator)  # cassification_experiment

load_model: Loading model: BTCUSDT_xgboost_SL_2.0_RT_720_RPL_24_3


In [7]:
for nf in numeric_features.copy():
    for i in range(1, regression_times + 1):
        col = nf + "_" + str(i)
        test_data[col] = test_data[nf].shift(i)
        numeric_features.append(col)

test_data.dropna(inplace=True)
test_data

,open_time,open,high,low,close,volume,rsi,open_1,open_2,open_3,...,rsi_1431,rsi_1432,rsi_1433,rsi_1434,rsi_1435,rsi_1436,rsi_1437,rsi_1438,rsi_1439,rsi_1440
48435,2023-03-02 14:00:00,23309.98,23349.21,23195.90,23305.46,19651.20803,38.20,23354.61,23414.73,23375.22,...,70.53,67.18,69.47,69.00,68.71,66.41,63.51,60.15,58.49,54.11
48436,2023-03-02 15:00:00,23305.46,23383.83,23274.08,23361.11,15400.28680,42.95,23309.98,23354.61,23414.73,...,58.13,70.53,67.18,69.47,69.00,68.71,66.41,63.51,60.15,58.49
48437,2023-03-02 16:00:00,23361.10,23387.84,23250.24,23279.98,11198.50222,38.32,23305.46,23309.98,23354.61,...,50.31,58.13,70.53,67.18,69.47,69.00,68.71,66.41,63.51,60.15
48438,2023-03-02 17:00:00,23279.45,23319.36,23231.29,23268.91,9066.42684,37.72,23361.10,23305.46,23309.98,...,56.48,50.31,58.13,70.53,67.18,69.47,69.00,68.71,66.41,63.51
48439,2023-03-02 18:00:00,23269.02,23458.94,23254.31,23441.14,12172.80560,50.63,23279.45,23361.10,23305.46,...,69.66,56.48,50.31,58.13,70.53,67.18,69.47,69.00,68.71,66.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52915,2023-09-04 21:00:00,25836.68,25842.23,25631.21,25677.71,1856.10935,30.14,25893.19,25915.66,25907.47,...,53.46,55.18,56.14,55.23,48.25,44.47,46.20,45.06,41.51,39.42
52916,2023-09-04 22:00:00,25677.72,25798.54,25632.61,25775.01,839.81115,41.63,25836.68,25893.19,25915.66,...,48.23,53.46,55.18,56.14,55.23,48.25,44.47,46.20,45.06,41.51
52917,2023-09-04 23:00:00,25775.02,25874.97,25764.02,25826.02,1283.81847,46.59,25677.72,25836.68,25893.19,...,45.27,48.23,53.46,55.18,56.14,55.23,48.25,44.47,46.20,45.06
52918,2023-09-05 00:00:00,25826.03,25844.48,25737.69,25768.49,860.37518,42.23,25775.02,25677.72,25836.68,...,49.00,45.27,48.23,53.46,55.18,56.14,55.23,48.25,44.47,46.20


In [8]:
predict = experiment.predict_model(model, data=test_data)
predict = predict.sort_values(date_features)

In [9]:
def test_trading_crypto_v3(data: pd.DataFrame, start_date, end_date, value: float, stop_loss = 3.0):
  _data = data.copy()
  _data.index = _data['open_time']
  _data = _data[(_data.index >= start_date) & (_data.index <= end_date)]
  saldo = value
  operacao = ''
  comprado = False
  valor_compra = 0 
  valor_venda = 0
  diff = 0.0
  
  operacao_compra = ''
  for row_nu in range(1, _data.shape[0]):
    open_time = pd.to_datetime(_data.iloc[row_nu:row_nu+1]['open_time'].values[0]).strftime("%Y-%m-%d %Hh")
    type(open_time)
    operacao = _data.iloc[row_nu:row_nu+1]['prediction_label'].values[0]
    #print(operacao)
    if (operacao.startswith('SOBE') or operacao.startswith('CAI')) and not comprado:
      operacao_compra = operacao
      valor_compra = round(_data.iloc[row_nu:row_nu+1]['close'].values[0], 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Compra: {valor_compra}')
      comprado = True

    if comprado:
      diff = 100 * (_data.iloc[row_nu:row_nu+1]['close'].values[0] - valor_compra) / valor_compra
      #print(f'[{row_nu}][{operacao_compra}][{open_time}] Diff ==> {round(diff,2)}% - Comprado: {comprado}')
    
    if (abs(diff) >= stop_loss) and comprado:
      valor_venda = round(_data.iloc[row_nu:row_nu+1]['close'].values[0],2)
      if operacao_compra.startswith('SOBE'):
        saldo += round(saldo * (diff/100), 2)
      else:
        saldo += round(saldo * (-diff/100), 2)
      print(f'[{row_nu}][{operacao_compra}][{open_time}] => Venda: {valor_venda} => Diff: {round(diff,2)}% ==> Saldo: {saldo}')
      comprado = False

  print(f'Saldo: {saldo}')
  return saldo

print('Min Data: ', predict['open_time'].min())
print('Max Data: ', predict['open_time'].max())
test_trading_crypto_v3(predict, '2023-01-01', predict['open_time'].max(), 100.0, stop_loss)

Min Data:  2023-03-02 14:00:00
Max Data:  2023-09-05 01:00:00
[1][CAI_2.0][2023-03-02 15h] => Compra: 23361.109375
[11][CAI_2.0][2023-03-03 01h] => Venda: 22150.310546875 => Diff: -5.18% ==> Saldo: 105.18
[111][CAI_2.0][2023-03-07 05h] => Compra: 22460.669921875
[135][CAI_2.0][2023-03-08 05h] => Venda: 21954.98046875 => Diff: -2.25% ==> Saldo: 107.55000000000001
[140][CAI_2.0][2023-03-08 10h] => Compra: 22111.25
[163][CAI_2.0][2023-03-09 09h] => Venda: 21641.580078125 => Diff: -2.12% ==> Saldo: 109.83000000000001
[164][CAI_2.0][2023-03-09 10h] => Compra: 21648.4296875
[172][CAI_2.0][2023-03-09 18h] => Venda: 21012.890625 => Diff: -2.94% ==> Saldo: 113.05000000000001
[173][CAI_2.0][2023-03-09 19h] => Compra: 20854.9296875
[174][CAI_2.0][2023-03-09 20h] => Venda: 20116.66015625 => Diff: -3.54% ==> Saldo: 117.05000000000001
[175][CAI_2.0][2023-03-09 21h] => Compra: 20213.609375
[188][CAI_2.0][2023-03-10 10h] => Venda: 19621.66015625 => Diff: -2.93% ==> Saldo: 120.48000000000002
[189][SOBE

1430.0500000000002